In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Veriyi yükle
train_df = pd.read_csv('/kaggle/input/playground-series-s3e17/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e17/test.csv')

# Temel bilgileri göster
print("EĞİTİM VERİSİ BOYUTU:", train_df.shape)
print("TEST VERİSİ BOYUTU:", test_df.shape)

In [ ]:
# Eğitim verisinin ilk 5 satırı
train_df.head()

In [ ]:
# Veri tipleri
train_df.dtypes

In [ ]:
# Boş değer kontrolü
print("EĞİTİM VERİSİ - BOŞ DEĞERLER:")
print(train_df.isnull().sum())

In [ ]:
# Temel istatistikler
train_df.describe()

In [ ]:
# Hedef değişken dağılımı
print("Machine Failure Dağılımı:")
print(train_df['Machine failure'].value_counts())
print("\nYüzde Dağılım:")
print(train_df['Machine failure'].value_counts(normalize=True))

In [ ]:
# Sütun isimlerini temizle
def clean_column_names(df):
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('[\[\]]', '', regex=True)
    return df

train_df = clean_column_names(train_df)
test_df = clean_column_names(test_df)

print("Yeni sütun isimleri:")
print(train_df.columns.tolist())

In [ ]:
# Temizlenmiş verinin ilk 5 satırı
train_df.head()

In [ ]:
# Hedef değişken dağılımı grafiği
plt.figure(figsize=(8, 6))
sns.countplot(data=train_df, x='Machine_failure')
plt.title('Machine Failure Dağılımı')
plt.xlabel('Machine Failure (0: Normal, 1: Arıza)')
plt.ylabel('Sayı')
plt.show()

In [ ]:
# Makine tiplerinin dağılımı
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
train_df['Type'].value_counts().plot(kind='bar')
plt.title('Makine Tiplerinin Dağılımı')
plt.xlabel('Type')
plt.ylabel('Sayı')
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
type_failure = train_df.groupby('Type')['Machine_failure'].mean()
type_failure.plot(kind='bar')
plt.title('Makine Tipine Göre Arıza Oranı')
plt.xlabel('Type')
plt.ylabel('Arıza Oranı')
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Sayısal değişkenlerin listesi
numerical_cols = ['Air_temperature_K', 'Process_temperature_K', 'Rotational_speed_rpm', 
                  'Torque_Nm', 'Tool_wear_min']

# Sayısal değişkenlerin dağılımı
plt.figure(figsize=(15, 10))

for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    plt.hist(train_df[col], bins=50, alpha=0.7, color='skyblue')
    plt.title(f'{col} Dağılımı')
    plt.xlabel(col)
    plt.ylabel('Frekans')

plt.tight_layout()
plt.show()

In [ ]:
# Boxplot ile aykırı değerler
plt.figure(figsize=(15, 10))

for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    plt.boxplot(train_df[col])
    plt.title(f'{col} Boxplot')
    plt.ylabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# Hedef değişkene göre sayısal değişkenlerin dağılımı
plt.figure(figsize=(15, 10))

for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(data=train_df, x='Machine_failure', y=col)
    plt.title(f'{col} - Machine Failure')
    plt.xlabel('Machine Failure')

plt.tight_layout()
plt.show()

In [ ]:
# Korelasyon matrisi
plt.figure(figsize=(12, 10))
correlation_matrix = train_df[numerical_cols + ['Machine_failure']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Korelasyon Matrisi')
plt.show()

In [ ]:
# Yeni özellikler oluşturma
def create_features(df):
    # Sıcaklık farkı
    df['Temperature_difference_K'] = df['Process_temperature_K'] - df['Air_temperature_K']
    
    # Güç hesaplama (Tork * Hız)
    df['Power_W'] = df['Torque_Nm'] * df['Rotational_speed_rpm']
    
    # Takım aşınma oranı
    df['Tool_wear_rate'] = df['Tool_wear_min'] / (df['Rotational_speed_rpm'] + 1)  # 0'a bölme hatası için +1
    
    # Sıcaklık oranı
    df['Temperature_ratio'] = df['Process_temperature_K'] / df['Air_temperature_K']
    
    # Hız bölü tork
    df['Speed_to_torque_ratio'] = df['Rotational_speed_rpm'] / (df['Torque_Nm'] + 1)
    
    return df

# Özellikleri oluşturma
train_df = create_features(train_df)
test_df = create_features(test_df)

print("Yeni oluşturulan özellikler:")
new_features = ['Temperature_difference_K', 'Power_W', 'Tool_wear_rate', 'Temperature_ratio', 'Speed_to_torque_ratio']
print(new_features)

In [ ]:
# Yeni özelliklerin dağılımı
plt.figure(figsize=(15, 12))

for i, col in enumerate(new_features, 1):
    plt.subplot(3, 2, i)
    plt.hist(train_df[col], bins=50, alpha=0.7, color='lightgreen')
    plt.title(f'{col} Dağılımı')
    plt.xlabel(col)
    plt.ylabel('Frekans')

plt.tight_layout()
plt.show()

In [ ]:
# Yeni özelliklerin hedef değişkenle ilişkisi
plt.figure(figsize=(15, 12))

for i, col in enumerate(new_features, 1):
    plt.subplot(3, 2, i)
    sns.boxplot(data=train_df, x='Machine_failure', y=col)
    plt.title(f'{col} - Machine Failure')
    plt.xlabel('Machine Failure')

plt.tight_layout()
plt.show()

In [ ]:
# Type değişkeninin one-hot encoding
train_df_encoded = pd.get_dummies(train_df, columns=['Type'], prefix='Type')
test_df_encoded = pd.get_dummies(test_df, columns=['Type'], prefix='Type')

print("Encoding sonrası sütunlar:")
print([col for col in train_df_encoded.columns if 'Type_' in col])

In [ ]:
# Gerekli kütüphaneleri içe aktar
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report
import xgboost as xgb

# Özellikleri ve hedef değişkeni belirle
feature_columns = [
    'Air_temperature_K', 'Process_temperature_K', 'Rotational_speed_rpm',
    'Torque_Nm', 'Tool_wear_min', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF',
    'Temperature_difference_K', 'Power_W', 'Tool_wear_rate', 
    'Temperature_ratio', 'Speed_to_torque_ratio',
    'Type_H', 'Type_L', 'Type_M'
]

X = train_df_encoded[feature_columns]
y = train_df_encoded['Machine_failure']

print("Kullanılan özellikler:")
print(feature_columns)
print(f"\nÖzellik matrisi boyutu: {X.shape}")
print(f"Hedef değişken boyutu: {y.shape}")

In [ ]:
# Veriyi böl
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Validasyon seti boyutu: {X_val.shape}")
print(f"Eğitim seti hedef dağılımı: {y_train.value_counts(normalize=True)}")
print(f"Validasyon seti hedef dağılımı: {y_val.value_counts(normalize=True)}")

In [ ]:
# Özellik ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Test seti için de ölçeklendirme
X_test = test_df_encoded[feature_columns]
X_test_scaled = scaler.transform(X_test)

print("Ölçeklendirme tamamlandı")
print(f"Eğitim seti ölçeklendirilmiş boyut: {X_train_scaled.shape}")

In [ ]:
# Random Forest modeli
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'  # Dengesiz sınıf için
)

# Modeli eğit
rf_model.fit(X_train, y_train)

# Tahminler
y_pred_proba = rf_model.predict_proba(X_val)[:, 1]

# AUC skoru
auc_score = roc_auc_score(y_val, y_pred_proba)
print(f"Random Forest AUC Score: {auc_score:.4f}")

In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Feature Importance grafiği
plt.figure(figsize=(10, 8))
sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
plt.title('Top 10 Feature Importance (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print("En Önemli 10 Özellik:")
print(feature_importance.head(10))

In [ ]:
# XGBoost modeli
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    scale_pos_weight=10  # Dengesiz sınıf için
)

# Modeli eğit
xgb_model.fit(X_train, y_train)

# Tahminler
y_pred_proba_xgb = xgb_model.predict_proba(X_val)[:, 1]

# AUC skoru
auc_score_xgb = roc_auc_score(y_val, y_pred_proba_xgb)
print(f"XGBoost AUC Score: {auc_score_xgb:.4f}")
print(f"Random Forest AUC Score: {auc_score:.4f}")

In [ ]:
# İki modeli karşılaştırma
models_comparison = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost'],
    'AUC_Score': [auc_score, auc_score_xgb]
})

plt.figure(figsize=(8, 6))
sns.barplot(data=models_comparison, x='Model', y='AUC_Score')
plt.title('Model Karşılaştırması - AUC Score')
plt.ylabel('AUC Score')
plt.ylim(0.8, 1.0)
for i, v in enumerate(models_comparison['AUC_Score']):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.show()

print(models_comparison)

In [ ]:
# En iyi modeli seç (burada XGBoost varsayalım)
best_model = xgb_model if auc_score_xgb > auc_score else rf_model

# Test seti için tahminler
test_predictions = best_model.predict_proba(X_test)[:, 1]

# Submission dosyası oluşturma
submission = pd.DataFrame({
    'id': test_df['id'],
    'Machine failure': test_predictions
})

print("Submission dosyası ilk 5 satırı:")
print(submission.head())

In [ ]:
# Submission dosyasını kaydet
submission.to_csv('submission.csv', index=False)
print("Submission dosyası kaydedildi!")
print(f"Submission dosyası boyutu: {submission.shape}")

In [ ]:
# Test seti tahminlerinin dağılımı
plt.figure(figsize=(10, 6))
plt.hist(test_predictions, bins=50, alpha=0.7, color='purple')
plt.title('Test Seti Tahminlerinin Dağılımı')
plt.xlabel('Tahmin Edilen Arıza Olasılığı')
plt.ylabel('Frekans')
plt.axvline(x=0.5, color='red', linestyle='--', alpha=0.7, label='0.5 Eşiği')
plt.legend()
plt.show()

print(f"Tahminlerin istatistikleri:")
print(f"Min: {test_predictions.min():.4f}")
print(f"Max: {test_predictions.max():.4f}")
print(f"Ortalama: {test_predictions.mean():.4f}")
print(f"Medyan: {np.median(test_predictions):.4f}")

In [ ]:
# Hiperparametre optimizasyonu için örnek (Grid Search)
from sklearn.model_selection import GridSearchCV

# Küçük bir grid search örneği (hızlı olması için sınırlı parametre)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'min_samples_split': [2, 5]
}

# Küçük bir örnek üzerinde çalıştır (hızlı olması için)
rf_small = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_small.fit(X_train.head(1000), y_train.head(1000))  # Küçük örnek

print("Hiperparametre optimizasyonu için hazır!")
print("Tam optimizasyon için daha fazla zaman gerekir.")

In [ ]:
print("🏆 PROJE ÖZETİ 🏆")
print("="*50)
print(f"📊 Veri Seti Boyutu: {train_df.shape[0]} satır, {train_df.shape[1]} sütun")
print(f"🎯 Hedef Değişken Dengesi: {y.mean():.3f} arıza oranı")
print(f"⚙️  Oluşturulan Özellik Sayısı: {len(feature_columns)}")
print(f"📈 En İyi Model AUC Skoru: {max(auc_score, auc_score_xgb):.4f}")
print(f"📁 Submission dosyası hazır: submission.csv")
print("="*50)
print("✅ Proje tamamlandı!")

In [ ]:
# Modeli kaydet
import joblib
joblib.dump(rf_model, 'model.pkl')
print("Model başarıyla kaydedildi!")